In [ ]:
import pandas as pd

def get_all_info(DF):
    
    for idx, row in DF.iterrows():
        d = row.to_dict()
        if d['Platform'] == 'all':
            return d
        
def get_Info(d):
    L = []
    for i in ['T', 'D', 'M', 'I']:
        info = {k:v for k, v in d.items() if i in k}
        # print(info)

        x = {k.replace(i+ '_', '').replace(i, 'GroupMember'): v for k, v in info.items()}
        x['Group'] = i
        L.append(x)
        
    Report = pd.DataFrame(L)
    Report = Report[['Group', 'GroupMember', 'recentscore', 'profilescore', 'recentrvw/allrvw']]
    return Report


T2Full = {
    'T': 'Physician Having Recent Reviews', 
    'D': 'Lower Recent Score than Profile Score',
    'M': 'Similar Recent Score to Profile Score',
    'I': 'Higher Recent Score to Profile Score'
}

def get_clean_result(d):
    Report = get_Info(d)

    L = []
    for idx, row in Report.iterrows():
        d = row.to_dict()
        if d['Group'] == 'T':
            d['Name'] = T2Full[d['Group']] + ' (' + d['GroupMember'].split(' ')[0] + ')'
        else:
            d['Name'] = T2Full[d['Group']] + ' ' + d['GroupMember'].split(' ')[1] 
        d['Group'] = T2Full[d['Group']]
        d['Group Member'] = int(d['GroupMember'].split(' ')[0])
        d['Recent Score'] = float(d['recentscore'].split(' ')[0])
        d['Profile Score'] = float(d['profilescore'].split(' ')[0])
        rctrvw, rvw = d['recentrvw/allrvw'].split(' ')[0].split('/')
        d['RctRvw/Physican'] = round(int(rctrvw) / d['Group Member'], 2)
        d['Rvw/Physican'] = round(int(rvw) / d['Group Member'], 2)
        d.pop('Group Member')
        L.append(d)

    Report = pd.DataFrame(L)
    
    return Report



In [ ]:
# feasible_physician_name = r'$\bfPhysician$ $\bf(PR)$: having both $\bfP$rior and $\bfR$ecent Reviews'
# feasible_physician_name = r'$\bfPhysician$: having both $\bfP$rior and $\bfR$ecent Reviews'
# feasible_physician_name = r'Physician: having both $\bfP$rior and $\bfR$ecent Reviews'
feasible_physician_name = r'Physicians with both Recent and Past Reviews'


Report_Dict = {}
path = 'Output/RecentVSProfile/Review0-MinTxt0_RctY3_PastRvw1_Report.csv'
DF = pd.read_csv(path)
d = get_all_info(DF)
Report = get_clean_result(d)
# Report.to_clipboard()
Report_Dict[feasible_physician_name] = Report

path = 'Output/RecentVSProfile/Review10-MinTxt0_RctY3_PastRvw1_Report.csv'
DF = pd.read_csv(path)
d = get_all_info(DF)
Report = get_clean_result(d)
# Report.to_clipboard()
Report_Dict[feasible_physician_name.split(' with')[0] + ' with 10+ reviews'] = Report


path = 'Output/RecentVSProfile/Review20-MinTxt0_RctY3_PastRvw1_Report.csv'
DF = pd.read_csv(path)
d = get_all_info(DF)
Report = get_clean_result(d)
# Report.to_clipboard()
Report_Dict[feasible_physician_name.split(' with')[0] + ' with 20+ reviews'] = Report



path = 'Output/RecentVSProfile/Review50-MinTxt0_RctY3_PastRvw1_Report.csv'
DF = pd.read_csv(path)
d = get_all_info(DF)
Report = get_clean_result(d)
# Report.to_clipboard()
Report_Dict[feasible_physician_name.split(' with')[0] + ' with 50+ reviews'] = Report



In [ ]:
[i for i in Report_Dict]

In [ ]:
Report_Dict['Physicians with both Recent and Past Reviews']

In [ ]:
18343 

In [ ]:
100 - 58.46

In [ ]:
import numpy as np

def get_report_info(title, Report):
    
    group1 = Report['profilescore'].values
    group1 = [float(i.split('(')[0]) for i in group1] # profilescore
    group1_label = 'Profile Score'
    # print(group1)
    # print(group1_label)

    group2 = Report['recentscore'].values
    group2 = [float(i.split('(')[0]) for i in group2] # profilescore
    group2_label = 'Recent Score'
    # print(group2)
    # print(group2_label)

    group_members = Report['GroupMember'].values
    # '{:,}'.format(value)
    g0 = group_members[0]
    # label0 = 'Physician Sample\n' + r'$\bf({:,})$'.format(int(g0.split('(')[0].strip()))
    label0 = 'Physicians\n' + r'$\bf({:,})$'.format(int(g0.split('(')[0].strip()))

    g1 = group_members[1]
    # label1 = 'Group: Lower Recent Rating than Profile Rating ({}'.format(g1.split('(')[1].strip())
    label1 = 'Lower Group\n' + r'$\bf({}$'.format(g1.split('(')[1].strip()).replace('%', '\%')

    g2 = group_members[2]
    label2 = 'Similar Group\n' + r'$\bf({}$'.format(g2.split('(')[1].strip()).replace('%', '\%')

    g3 = group_members[3]
    label3 = 'Higher Group\n' + r'$\bf({}$'.format(g3.split('(')[1].strip()).replace('%', '\%')

    labels = [label0, label1, label2, label3]
    x = np.array(list(range(len(labels))))

    return group1, group1_label, group2, group2_label, labels, x


title = feasible_physician_name
Report = Report_Dict[title]

group1, group1_label, group2, group2_label, labels, x = get_report_info(title, Report)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_grouped_bar(ax, x, 
                     group1, group2, group1_label, group2_label, 
                     labels, width, title):
    rects1 = ax.bar(x - width*0.55, group1, width, label = group1_label, color = '#4285F4')
    rects2 = ax.bar(x + width*0.55, group2, width, label = group2_label, color = '#EA4336')
    # Add some text for labels, title and custom x-axis tick labels, etc.
    
    ax.set_title(title)
    ax.set_xticks(x, labels)
    ax.legend()
    ax.bar_label(rects1, padding=1, fontsize = 12)
    ax.bar_label(rects2, padding=1, fontsize = 12)
    
    ax.set_ylabel('Rating Score', fontsize = 12)
    # ax.y xticks(np.arange(1, 5, 1))
    # ax.yaxis.set_ticks()
    ax.set_yticks(np.arange(1, 6, 1))
    # ax.set_yticklabels(fontsize = 12)
    ax.set_ylim(0.9, 5.1)
    
    return ax


# fig, ax = plt.subplots()

fig = plt.figure(figsize=(15, 10), dpi=100)
fig.tight_layout() 

width = 0.35
# Adds subplot on position 1
axes = fig.subplots(2, 2)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                    wspace=0.15, hspace=0.3)


panel = 'ABCD'
for idx, ax in enumerate(fig.get_axes()):
    
    title = [i for i in Report_Dict][idx]
    
    Report = Report_Dict[title]
    
    #if idx >= 2: title = '\n' + title
    group1, group1_label, group2, group2_label, labels, x = get_report_info(title, Report)
    ax = plot_grouped_bar(ax, x, 
                          group1, group2, group1_label, group2_label, 
                          labels, width, r'$\bfPanel ({})$: '.format(panel[idx]) + title)
    
    
plt.show()
    

In [ ]:
fig.savefig('Figure1_v2_a.eps', format = 'eps', dpi=500)